# Implementation

In [1]:
pip install easygui

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import easygui as eg
from pprint import pprint as pprint
from pdb import set_trace as bp

In [2]:
class Dice():
    '''
    You should define a class called Dice. This class should hold the current value of two dice. (10 points)
        The Dice should have a method roll that rolls the 6-sided dice
    '''
    def __init__(self):
        self.values = np.random.randint(1,high=6,size=(2))
        
    def roll(self):
        self.values = np.random.randint(1,high=6,size=(2))
        return self.values

#### test
dice = Dice()
print(dice.values)
dice.roll()
print(dice.values)

[4 4]
[4 4]


In [3]:
class Table(Dice):
    '''
    You should define a class Table (10 points)
        This class should inherit the Dice which are associated with the table.
        The table has a state point that determines if the point has been set or not.
    ''' 
    def __init__(self,dice=Dice(),point=False):
        super(Table,self).__init__()
        self.point = point
        
    def legit_decorator(self,condition_func):
        '''
        performs int check, then performs condition_func check
        '''
        input_ = False
        while input_==False:
            try: input_ = int(input('Input amount: '))
            except: 
                print('Please input an integer dollar amount')
                input_ = False
                continue
            input_ = condition_func(input_)
        return input_
        
#### test
table = Table()
print(table.values)
print(table.point)

[2 1]
False


In [4]:
# Define a Class for the player - note the player belongs to the table and thus should inherit from it. (15 points)

class Player(Table):
    def __init__(self,table=Table,name=None):
        super(Player,self).__init__()
        # In the initialization, the code should ask the user to input a name and save the player’s name.
        # Hint: you can use the built-in method input.
        self.name = input('Please input your name: ')
        '''
        The table only allows bets of dollars, cents cannot be used. Figure out a way to enforce this.
            The script should tell the player they cannot have a bankroll that is not convertible into a dollar amount.
            You can use error handling try and except for this
            If the user enters a value that is not acceptable - prompt the user to enter their bankroll again. 
                Hint: This might be an excellent place to use a decorator
            The script should continue to ask the user for their bankroll until a valid number is entered.
        '''
        print('Please input bankroll: ')
        @self.legit_decorator
        def legit(x):
            if x==0:
                print('Please input a non-zero integer')
                return False
            else:
                return x
        self.bankroll = legit

In [ ]:
# Define an Class for bets (65 Points - you can earn 80)
# The bets should belong to the player and thus, it would make sense to inherit from them.
# Within the bets class you should keep track of all the possible bets - 
    # pass line, do not pass, odds_bet; 
# it might also be good to store the maximum allowable odds bet in the current state.
class Bets(Player):
    def __init__(self,player):
        # self = player
        super(Bets,self).__init__(player)
        # super().__init__()
        self.bets = {'pass_line': [],
                     'do_not_pass': [],
                     'pass_odds': [],
                     'do_not_pass_odds': [],
                     '4_odds': [],
                     '5_odds': [],
                     '6_odds': [],
                     '8_odds': [],
                     '9_odds': [],
                     '10_odds': []
                    }
        self.won = 0
        self.lost = 0
        self.max_bet = self.bankroll
        self.out_of_money = False
        
    # Build methods that print the bet made, bet won, and bets lost. 
        # These should identify the amount of money won or lost and the remaining balance. (5 points)
    def bet_made(self,bet):
        placed_bets = [(key,self.bets[key]) for key in self.bets if len(self.bets[key])>0]
        print(f'Current bets: {placed_bets}')
        print(f'Remaining Balance: {self.bankroll}\n')
    def bet_won(self,bet):
        print(f'{bet} Won: {self.won}')
        print(f'Remaining Balance: {self.bankroll}')
    def bet_lost(self,bet):
        print(f'{bet} Lost: {self.lost}')
        print(f'Remaining Balance: {self.bankroll}')
        
        
    # Implement a method insufficient_funds that is called if the player enters a bet greater than their bankroll. 
    def insufficient_funds(self,bet_amount):
        # This should print the name of the player, and that they have insufficient funds to place a bet (list the amount)
        # on the bet position. (5 points)
        # It should prompt the player to make another bet
        try:  # if you have no money left, exit
            1/round(self.bankroll)
        except: 
            print('You are out of money.')
            self.out_of_money = True
            return True
        if bet_amount>self.max_bet:
            print(f'Exceeded allowed odds bet. Please input a bet at or below {self.max_bet}.')
            return True
        # try:  # if you have no money for odds
        #     1/round(self.bankroll)
        # except: 
        #     print('You are out of money.')
        #     self.out_of_money = True
        #     return True
        if bet_amount>self.bankroll:
            print(f'Hi {self.name}, Not enough for {bet_amount}. Bet at or below {self.bankroll}.')
            return True
            
        
    # Implement a method pass_line - this method should allow the player to set a bet on the pass line only if the point
    # has not been established and the player has enough funds for their bet. Make sure to update the bankroll (5 points)
    def pass_line(self,bet_amount):
        if self.point==False and not self.insufficient_funds(bet_amount):
            self.bankroll-=bet_amount
            self.bets['pass_line'].append(bet_amount)

            
    # Implement a method do_not_pass - - this method should allow the player to set a bet on the do not pass line only 
    # if the point has not been established and the player has enough funds for their bet. 
    # Make sure to update the bankroll (5 points)
    def do_not_pass(self,bet_amount):
        if self.point==False and not self.insufficient_funds(bet_amount):
            self.bankroll-=bet_amount
            self.bets['do_not_pass'].append(bet_amount)

            
    # Implement a method betting_turn that starts the betting phase (10 points)
    def betting_turn(self): # starts the coming out phase
        # Ask the user if they want to place any bets. Accepts a yes or no answer - case insensitive
        bet = eg.boolbox('Would you like to place a bet?',choices=('Yes','No'))  
        if not bet: 
            print('No bets placed')
            return
        else:
            # Ask the user where they want to bet - there are two options, pass line or do not pass line. 
            # – Make sure this is case insensitive. You should save where you are placing your bets.
            bet = eg.choicebox('Place bet: ',choices=('pass_line','do_not_pass'))
            while bet[0]==None:
                bet = eg.choicebox('Must place a bet here: ',choices=('pass_line','do_not_pass'))


            # Call an ingest_bet method which we will define later. This method should return the bet amount.
            # If the player inputs an invalid bet name, call this method again so the player can make a valid bet
            self.bet_made(self.ingest_bet(bet))
            return bet
                    
              
    # Implement a method ingest_bet this method should: (10 points)
    def ingest_bet(self,bet):
        '''
        bet is a string for the key value
        '''
        # Prompt the user for a bet - enforce that this is an integer value – 
            # if not, tell the player that a non-integer value cannot be used and prompt them to make another wager.
        print(f'Input an integer non-zero dollar bet on {bet}: ')
        
        @self.legit_decorator
        def legit(x):
            # Check to make sure the player did not bet more money than they have – 
            # If they did, tell them they cannot and tell them the maximum value they can bet
            if self.insufficient_funds(x):
                return False
            # Check to see if the player made a zero bet – 
            # if they did, prompt them that it is not allowed and ask them to make another wager
            if x==0:
                print('Please input a non-zero bet.')
                return False
            # make sure odds bet isn't too large
            if self.point:
                if len(self.bets['pass_line'])==0:
                    print('Cannot bet odds')
                    return False
                
            # self.bets[bet].append(x)
            return x
                    
        bet_amount = legit
        if bet=='pass_line': self.pass_line(bet_amount)
        elif bet=='do_not_pass': self.do_not_pass(bet_amount)
        else: ## odds bet
            self.bankroll-=bet_amount
            self.bets[bet].append(bet_amount)
        # Make sure this method returns the bet amount.
        return bet_amount

        
    # Build a method called shooter that checks if any there are any active bets - 
        # if there are, it should roll the dice and evaluate payouts (5 points)
    def shooter(self):
        if any([len(self.bets[key])>0 for key in self.bets]): # if there are active bets
            self.roll()
            if self.point==False: # coming out
                print(f'Rolled {self.values}: {self.values.sum()}')
                self.payout()
            else:
                print(f'Rolled {self.values}: {self.values.sum()}')
                self.odds()
                self.payout()
        else: # test if we want to start betting
            print('No active bets')
            if self.betting_turn()==None:
                return
            self.shooter()
            self.payout()
            

    # Implement a method Bet_loser that prints that save the losing bet type and amount. 
    def bet_loser(self,bet):
        self.lost += sum(self.bets[bet])
        for key in self.bets:
            self.bets[key]=[]
        # This should then call the bets_lost method (5 points)
        self.bet_lost(bet)

                                      
    # Implement a method Bet_winner this should: (10 points)
    def bet_winner(self,bet):
        # Save the winning bet type and amount
        amount = sum(self.bets[bet])
        self.won += amount
        # Add the winning amount to your bankroll
        self.bankroll += 2*amount
        # Remove the odds bets - as these are only allowed once the point is established. They should be returned to the bankroll.
        for key,val in self.bets.items():
            if '_odds' in key:
                if key!=bet and key!='do_not_pass': self.bankroll+=sum(val)
                self.bets[key]=[]
        # Call the method to print the winning bet
        self.bet_won(bet)
        self.bets[bet] = []

               
    # Implement a method Odds, this method, when called should: (5 points)
    def odds(self):
        # Check to see if an odds bet can be made. - There must be an existing pass-line bet by the player
        if len(self.bets['pass_line'])==0: 
            print('No bet on pass_line. Cannot bet odds')
            return None
        # Determine the maximum bet that can be placed. 
            # This is the lower of the bankroll or the allowable multiple of the pass-line bet set by the casino 3-4-5.
        if self.point==4 or self.point==10:
            self.max_bet = 3*sum(self.bets['pass_line'])
        elif self.point==5 or self.point==9:
            self.max_bet = 4*sum(self.bets['pass_line'])
        elif self.point==6 or self.point==8:
            self.max_bet = 5*sum(self.bets['pass_line'])
        current_odds = sum( [sum(val) for key,val in self.bets.items() if '_odds' in key] )
        self.max_bet -= current_odds
        # Print a statement providing the user with the maximum bet they can place, and their current bankroll
        print(f'Max allowed odds bet: {self.max_bet}. Current Bankroll: {self.bankroll}')
        
        if self.max_bet==0:
            print('You have reached the max odds bet and cannot bet odds anymore')
            return None
        if self.bankroll==0: 
            print('You are out of money')
            return None
    
        bet = eg.choicebox('Place odds bet: ',
                                       choices=('pass_odds','do_not_pass_odds',
                                                '4_odds','5_odds','6_odds','8_odds','9_odds','10_odds'))
        while bet==None:
            bet = eg.choicebox('Your must place odds bet: ',
                               choices=('pass_odds','do_not_pass_odds',
                                        '4_odds','5_odds','6_odds','8_odds','9_odds','10_odds'))
        # Call the ingest bet function to place an odds bet
        # Remove the bet amount from the current bankroll
        self.bet_made({self.ingest_bet(bet)})

        
    # Implement a method Payout. This method I will not provide a guideline for this, however, it should: (15 points)
    def payout(self):
        roll=self.values
        # Reconcile all bets based on outcomes
        if self.point==False: # come out phase
            if roll.sum()==2 or roll.sum()==3:
                self.bet_winner('do_not_pass')
                self.bet_loser('pass_line')
            elif roll.sum()==12:
                self.bet_winner('do_not_pass')
                self.bets['pass_line']=[]
            elif roll.sum()==7 or roll.sum()==11:
                self.bet_winner('pass_line')
                self.bet_loser('do_not_pass')
            else:
                self.point = roll.sum()  
                print(f'Point set to {self.point}')
                
        else: # points phase
            if roll.sum()==7: # game ends. do not pass and do not pass odds wins
                self.bet_winner('do_not_pass')
                self.bet_winner('do_not_pass_odds')
                self.bet_loser('pass_line')
                self.bet_loser('pass_odds')
                self.point = 0
                self.max_bet = self.bankroll

            elif roll.sum()==self.point: # game ends
                self.bet_winner('pass_line')
                self.bet_winner('pass_odds')
                self.bet_loser('do_not_pass')
                self.bet_loser('do_not_pass_odds')
                self.point = 0
                self.max_bet = self.bankroll

                    
            else: # continues
                if any( [len(self.bets[key])>0 for key in self.bets] ): # assign payout ammounts for any odds bets
                    if roll.sum()==4 or roll.sum()==10:
                        odd_amount = (1+2/1)*(sum(self.bets[f'{roll.sum()}_odds']))    
                    elif roll.sum()==5 or roll.sum()==9:
                        odd_amount = (1+3/2)*(sum(self.bets[f'{roll.sum()}_odds']))        
                    elif roll.sum()==6 or roll.sum()==8:
                        odd_amount = (1+6/5)*(sum(self.bets[f'{roll.sum()}_odds']))    
                    else: # rolling non interactive number
                        odd_amount = 0
                if odd_amount:
                    self.won += odd_amount
                    self.bankroll += odd_amount
                    self.bets[f'{roll.sum()}_odds']=[] # clear bet after adding
                    self.bet_won(f'{roll.sum()}_odds')

        # If there are no active bets, it should prompt the user to make a new bet or walk away with their remaining money
        if all( [len(self.bets[key])==0 for key in self.bets] ): 
            print('No active bets.')
            # If they do this, it should tell them how much they won or lost
            print(f'In total, Won: {self.won}\t Lost: {self.lost}\n')
            print(f'Remaining: {self.bankroll}')
            if self.bankroll==0:
                print('out of money')
                return
            if eg.boolbox('Make new bet?',choices=('Yes','No')):
                self.shooter()

        # If there are active bets, it should prompt them to roll the dice again
        else: 
            print('Bets remaining. Rolling again \n\n')
            self.odds()
            # if not self.point:
            #     self.betting_turn()
            self.shooter()

# Test

In [6]:
# define player1
player1 = Player()
print(player1.bankroll)
print(player1.name)
print(player1.values)
print(player1.point)

Please input your name:  x


Please input bankroll: 


Input amount:  10


10
x
[2 1]
False


In [7]:
# initialize bets 
bet = Bets(player1)
bet.shooter()

Please input your name:  x


Please input bankroll: 


Input amount:  10
